In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:,.2f}'.format

# Intro
As a first ML practice, I'm analyzing Mexico's COVID patients dataset to find any relation between medical conditions and death reate; and looking for a model that can predict the probability of a person to die from COVID.

# Dataset
This is the official dataset from Mexico's Secretariat of Health regarding confirmed COVID cases in public hospitals.
Let's see what we have here...

In [ ]:
general = pd.read_csv('/kaggle/input/covid19-mx/covid-19_general_MX.csv')
general.rename( columns={'Unnamed: 0' :'#'}, inplace=True )

print(general.shape)
display(general.head())
print(general.info())

## Clean Data, Select Features and Targets
* Parse values, filter NaNs.
* Choose only confirmed positive COVID cases.
* Select patient conditions as inputs: *EDAD* (age), *SEXO*, *DIABETES*, *EPOC*, *ASMA*, *INMUSUPR*, *HIPERTENSION*, *CARDIOVASCULAR*, *OBESIDAD*, *RENAL_CRONICA*, *TABAQUISMO*, *OTRA_CON* (other).
* Select complications and death as outputs: *MUERTE* (death), *NEUMONIA*, *UCI*, *INTUBADO*

In [ ]:
from sklearn.preprocessing import StandardScaler
features = general.copy()

# Parse dates
features['FECHA_INGRESO'] = pd.to_datetime(features['FECHA_INGRESO'])
features['FECHA_SINTOMAS'] = pd.to_datetime(features['FECHA_SINTOMAS'])
features['MUERTE'] = (features['FECHA_DEF'] != '9999-99-99').replace([True, False], [1, 0])
features['FECHA_DEF'].replace({'9999-99-99': '2099-01-01'}, inplace=True)
features['FECHA_DEF'] = pd.to_datetime(features['FECHA_DEF'])
features['DIAS'] = (features['FECHA_DEF'] - features['FECHA_SINTOMAS']).dt.days
features['DIAS_INCUBA'] = (features['FECHA_INGRESO'] - features['FECHA_SINTOMAS']).dt.days

# Select inputs and autputs
cond_headers = ['DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', 'OTRA_CON']
outcome_headers = ['NEUMONIA', 'INTUBADO', 'UCI', 'MUERTE']

# Format binary features
binary_headers = list(set(features.columns) - set(['index', 'EDAD']))
features[binary_headers] = features[binary_headers].replace({1: 1, 2: 0, 3: np.nan, 97: np.nan, 98: np.nan, 99: np.nan})
features.loc[:] = features.dropna()

#features['EDAD'] = StandardScaler().fit_transform(features['EDAD'].values.reshape(-1,1))

# Select only confirmed COVID cases, drop unused columns
infectados = features[features['RESULTADO'] == 1]
infectados = infectados.drop(columns=['SECTOR', 'ENTIDAD_UM', 'ENTIDAD_RES', 'TIPO_PACIENTE', 'NACIONALIDAD', 
                                'OTRO_CASO', 'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'RESULTADO', '#'])

feature_headers = list(set(infectados.columns) - set(outcome_headers))

display(infectados.info())

# Exploration
## Incidence of complications by medical condition

In [ ]:
fig, ax = plt.subplots(1, len(outcome_headers), figsize=(25,5))
for i, target in enumerate(outcome_headers):
    groups = infectados[cond_headers + [target, 'SEXO']].groupby([target, 'SEXO']).sum()
    groups = groups.unstack().unstack().reset_index(name='count')
    groups.replace('SEXO', {0:'male', 1:'female'}, inplace=True)

    alive = groups[groups[target]==0]
    dead = groups[groups[target]==1].copy()
    rate = dead['count'] / (dead['count'] + alive['count'].values)
    dead['rate']=rate

    sns.barplot(data=dead.sort_values('rate', axis='rows', ascending=False),
                ax=ax[i],
                x='level_0',
                y='rate',
                hue='SEXO',
                palette=sns.color_palette("BrBG", 2))
    ax[i].tick_params(labelrotation=60)
    ax[i].set_title('Percent of patients that resulted in ' + target)
    ax[i].set_xlabel('')
    ax[i].set_ylabel('')
plt.show()

* Complications on female patients are notably lower

## Death Rate

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 5))
ax[0].grid()
ax[1].grid()

# Days-from-symptoms histogram
data = infectados[infectados['MUERTE']==0]['DIAS_INCUBA']
sns.distplot(data, ax = ax[3], hist=True, kde=False, label="Non fatal cases").set_title('Days from symptoms to treatment')
data = infectados[infectados['MUERTE']==1]['DIAS_INCUBA']
sns.distplot(data, ax = ax[3], hist=True, kde=False, label="Fatal cases")
ax[3].set_xlabel('days')
ax[3].legend()
plt.sca(ax[3])
plt.xlim(0, 15)

# Days-to-death histogram
data = infectados[infectados['MUERTE']==1]['DIAS']
sns.distplot(data, ax = ax[0]).set_title('Days from symptoms to death')
ax[0].set_xlabel('days')
plt.sca(ax[0])
plt.xlim(0, 40)

# Age histogram
data = infectados['EDAD']
sns.distplot(data,
             hist=True, 
             kde=False, 
             ax=ax[1], 
             label='Total cases').set_title('Death distribution by age')
data = infectados[infectados['MUERTE']==1]['EDAD']
sns.distplot(data, 
             hist=True,
             kde=False,
             ax=ax[1],
             label='Deaths').set_title('Cases distribution by age')
ax[1].legend()
ax[1].set_xlabel('age')

# Death histogram
sns.lineplot(data=infectados.replace('SEXO', {0: 'Male', 1: 'Female'}),
             x='EDAD',
             y='MUERTE',
             hue='SEXO',
             ci=None,
             ax=ax[2]).set_title('Death rate by age and sex')
ax[2].set_xlabel('age')
ax[2].set_ylabel('')
plt.sca(ax[2])
plt.xlim(20, 80)
plt.ylim(0, .6)

plt.show()

* Again, female patients have a lower death rate
* Above the age of 60, reath rate rises considerably
* The time a patient spends with symptoms before getting to the hospital does not chance the chances of death

## Correlation Analysis

### Incidence of conditions among age groups

Most of the conditions have a presence too low to be considered, so only Diabetes, Hypertension and Obesity are analyzed.


In [ ]:
from sklearn.preprocessing import PowerTransformer, StandardScaler

def plot_death_rate(column, ax, scale=False):
    df = infectados.groupby(['EDAD', 'MUERTE']).mean().reset_index()
    
    plt.sca(ax)
    plt.xlim(20, 80)
#    plt.ylim(0, .15)
    if (scale):
        df[column] = PowerTransformer().fit_transform(df[column].values.reshape(-1,1))
    else: 
        ax.set_ylabel('Percent of ' + column)

    ax.scatter(data=df[df['MUERTE']==0], x='EDAD', y=column, label='non fatal')
    ax.scatter(data=df[df['MUERTE']==1], x='EDAD', y=column, label='fatal')
    ax.set_xlabel('age')
    ax.set_title(column + ' by age group')
    ax.legend()
    

headers = ['DIABETES', 'HIPERTENSION', 'OBESIDAD']
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
for i, condition in enumerate(headers):
    plot_death_rate(condition, ax[i])
plt.show()    


* Diabetes and Hypertension increase with age groups, while Obesity tends to center around 50's
* There seems to be a direct correlation between Diabetes, Hypertension and Age. So if Age is a major risk factor, maybe due to these conditions.
* Obesity seems to count towards death rate even in younger age groups

#### Correlation of other conditions to seath rate
These are the non principal conditions, data has been scaled

In [ ]:
headers = list(set(cond_headers) - set(headers))
fig, ax = plt.subplots(1, 7, figsize=(30, 5))
for i, condition in enumerate(headers):
    plot_death_rate(condition, ax[i], True)
plt.show()    

* Cardiovascular diseases, chronic renal deseases and immunosuppression seem to present a clustering pattern that may be exploited for a classification algorithm.

## Feature Selection
To try to predict patient death, these relevant features chosen:

* Age (discarded Hypertension and Diabetes as they all show direct correlation)
* Sex
* Obesity
* Chronic Renal
* Cardiovascular
* Immunosuppression

# Model Selection

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [ ]:
def grid_search_report(estimator, params, X_train, y_train):
    cv = GridSearchCV(estimator, params, cv=3).fit(X_train, y_train)
    results = pd.DataFrame(cv.cv_results_)
    test_scores = results[['split0_test_score', 'split1_test_score', 'split2_test_score']]#, 'split3_test_score', 'split4_test_score']]
    test_scores = test_scores.transpose()
    sns.barplot(data=test_scores)
    plt.xlabel('combinations')
    plt.ylabel('train score')
    plt.show()
    print('Best param combination: ', cv.best_params_)
    return cv

In [ ]:
def print_classification_results(estimator, X, y, plot_proba=False, debug=True):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    
    if (debug):
        print('\nPositives count: ' + str(len(positives)) + '; Negatives count: ' + str(len(negatives)), '\n')
        print('INPUT:\n')
        display(X_train.info())
        print('\nCONFUSION MATRIX:\n\n' + str(metrics.confusion_matrix(y_test, y_pred)))
        print('\nCLASSIFICATION REPORT:\n\n', metrics.classification_report(y_test, y_pred))

    y_proba = estimator.predict_proba(X_test)
    comparison = pd.DataFrame(columns=['EDAD', 'prediction', 'actual result'])
    comparison['EDAD'] = X_test['EDAD'].reset_index(drop=True)
    comparison['prediction'] = pd.DataFrame(y_proba)[1]
    comparison['actual result'] = y_test.reset_index(drop=True)

    if (plot_proba):
        plt.figure(figsize=(10, 10))
#        plt.xlim(20,85)
        sns.scatterplot(data=comparison, x='EDAD', y='prediction', hue='actual result')
        plt.ylabel('predicted probability for ' + target)
        plt.xlabel('normalized age')
        plt.show()
        
    return comparison

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import fbeta_score, make_scorer

def get_cv_results(estimator, X, y):
    scoring = {
            'f1': make_scorer(metrics.f1_score),
            'recall': make_scorer(metrics.recall_score),
            'accuracy': make_scorer(metrics.accuracy_score)
            }
    cv_results = cross_validate(estimator, X, y, scoring=scoring)
    mean_f1 = np.mean(cv_results['test_f1'])
    mean_accuracy = np.mean(cv_results['test_accuracy'])
    mean_recall = np.mean(cv_results['test_recall'])
    
    df = pd.DataFrame([[type(estimator).__name__, mean_accuracy, mean_recall, mean_f1]], columns=['Estimator', 'Accuracy Score', 'Recall Score', 'F1 Score'])
    return df

## Algorithm Comparison

In [ ]:
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import ensemble
from sklearn.decomposition import PCA

# Balance dataset
positives = features[features['RESULTADO']==1][features['MUERTE']==1]
negatives = features[features['RESULTADO']==1][features['MUERTE']==0]
min_samples = min(2000, len(positives), len(negatives))
negatives = negatives.sample(min_samples)
positives = positives.sample(min_samples)
balanced_data = pd.concat([negatives, positives])

# Split
X = balanced_data[['EDAD', 'SEXO', 'OBESIDAD', 'CARDIOVASCULAR', 'RENAL_CRONICA', 'INMUSUPR', 'DIAS_INCUBA']] # <- The features that make more sens to me
#X = balanced_data.drop(columns=outcome_headers + ['DIAS', '#', 'FECHA_SINTOMAS', 'FECHA_DEF', 'FECHA_INGRESO', 'OTRA_CON']) # <- All features but outcomes
y = balanced_data['MUERTE']

# Scale
X = pd.DataFrame(PowerTransformer().fit_transform(X), columns=X.columns)

# Reduce Dimensionality
#pca = PCA(n_components=16).fit(X)
#print(X.info(), '\n\nPCA:')
#print(np.cumsum(pca.explained_variance_ratio_))
#X = pca.transform(X) # <- It barely has any effect

# Grid Search
#grid_search_report(RadiusNeighborsClassifier(),{}, X, y)

# Report cross validation results
pd.options.display.float_format = '{:,.4f}'.format
df = pd.DataFrame()
df = df.append(get_cv_results(KNeighborsClassifier(n_neighbors=500), X, y))
df = df.append(get_cv_results(MLPClassifier(activation='logistic'), X, y))
df = df.append(get_cv_results(linear_model.LogisticRegression(), X, y))
df = df.append(get_cv_results(linear_model.RidgeClassifier(normalize=True), X, y))
df = df.append(get_cv_results(SVC(C=.01, kernel='poly', degree=1), X, y))
#df = df.append(get_cv_results(DecisionTreeClassifier(), X, y))
#df = df.append(get_cv_results(RadiusNeighborsClassifier(p=1, weights='distance', radius=10), X, y))
display(df.reset_index().sort_values(by='F1 Score', ascending=False))

selected_estimator = KNeighborsClassifier(n_neighbors=500)

## K-Neares Neighbor Classifier
This KNN Classifier is one of the best-performing classifiers tested. However, it's getting a **65% precision score** (not too good if you note that randomly you get a 50% change of guessing) and a **70% recall score**.
This accuracy means that if the model predicts someone could die from COVID, it would be true only 65% of the time.
The the recall score of 70% means that the other 30% of actual COVID deaths won't be predicted by the model.


In [ ]:
results = print_classification_results(selected_estimator, X, y, True)

# Final Notes
* Age-Hipertension-Diabetes seems to be the more influent feature on death rate. Sex plays its part too.
* There is no obvious influence in medical conditions upon death rate. Surprisingly Asthma and Smoking seem to represent the lower death rates.
* None of the tested models for classification with its parameters (up to 10 models tested), got pass 70% accuracy.

Improvements to data and models are welcome!

In [ ]:
# Imports for Deep Learning
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.models import Sequential

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = np.array(X_train).reshape(7,-1)
y_train = np.array(y_train).reshape(1,-1)
display(X_train.shape, y_train.shape)

model=Sequential([
                  Dense(7, input_shape=[7]),
                  Dense(1, activation="sigmoid")
                ])
display(model.summary())
model.compile()
model.fit(X_train, y)